In [74]:
import sqlite3
import pandas as pd
import os
from pathlib import Path
import numpy as np
import re
import openpyxl
import datetime
from datetime import datetime as dt
from datetime import date, timedelta
from pandas import ExcelWriter
from openpyxl import load_workbook

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [75]:
try:
    sqlite_connection = sqlite3.connect('test_sqlite_python.db')
    cursor = sqlite_connection.cursor()
    print("База данных создана и успешно подключена к SQLite")

    sqlite_select_query = "select sqlite_version();"
    cursor.execute(sqlite_select_query)
    record = cursor.fetchall()
    print("Версия базы данных SQLite: ", record)
    cursor.close()

except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)
finally:
    if (sqlite_connection):
        sqlite_connection.close()
        print("Соединение с SQLite закрыто")

База данных создана и успешно подключена к SQLite
Версия базы данных SQLite:  [('3.32.3',)]
Соединение с SQLite закрыто


In [76]:
path_to_test_xlsx = Path(os.getcwd()).parent.absolute()
path_to_test_xlsx = str(path_to_test_xlsx) + '\\'

In [77]:
cols_name = pd.read_excel(f'{path_to_test_xlsx}Наименования_столбцов_для_сборки_всех_приемов_онколога.xlsx', sheet_name = 0)
cols_name = cols_name.fillna('—')
cols_name_dict = {}
for _ in cols_name.columns.tolist():
    cols_name_dict[_] = [i for i in cols_name[_].values.tolist() if i != '—']
    
# читаем таблицу с крааткими намиенованиями ЦАОПов для ПРИЕМОВ ОНКОЛОГА
true_caop = pd.read_excel(f'{path_to_test_xlsx}СЛОВАРЬ_ЦАОП_2.xlsx')

In [78]:
print(f'Начало сборки: {dt.now()}')
start_reading = dt.now()


oncologist_consultations_visits_list = ['Первичная консультация +',
                                       'Повторная консультация +',
                                       'Повторная после ОК +',
                                       'Справки и направления +'
                                      ]

confirmed_diagnosis_visits_list = ['КО в рамках ДН +',
                                   'Выписка рецепта на ЛП +',
                                   'Снятие с учета +'
                                  ]

common_survey_visits_list = ['Общий осмотр']



oncologist_consultations_all_visits_dfs_list = []
confirmed_diagnosis_all_visits_dfs_list = []
common_survey_all_visits_dfs_list = []


# добавить столбец с датой выгрузки для проверки дублей

for _ in os.listdir(path_to_test_xlsx):
    if '.xlsx' in str(_):
        print(f'Читение: {_}')
        for w_sheet in load_workbook(path_to_test_xlsx + _, read_only=True).sheetnames:

            if w_sheet in oncologist_consultations_visits_list:
                print(w_sheet)
                target_df = pd.read_excel(path_to_test_xlsx + _,
                                    sheet_name = w_sheet,
                                    skiprows = 1,
                                    dtype=str)
                target_df['тип приема'] = w_sheet
    #             ######### дописать ИМЯ ФАЙЛА
                cols_quantity = target_df.shape[1]
                print(cols_quantity)
                try:
                    target_df.columns = cols_name_dict[cols_quantity]
                except:
                    print(f'Количество столбцов на листе {w_sheet} отличается от стандартного - необходимо изучить ошибку!')
                oncologist_consultations_all_visits_dfs_list.append(target_df)

            elif w_sheet in confirmed_diagnosis_visits_list:
                print(w_sheet)
                target_df = pd.read_excel(path_to_test_xlsx + _,
                                    sheet_name = w_sheet,
                                    skiprows = 1,
                                    dtype=str)
                target_df['тип приема'] = w_sheet
                cols_quantity = target_df.shape[1]
                print(cols_quantity)
                try:
                    target_df.columns = cols_name_dict[cols_quantity]
                except:
                    print(f'Количество столбцов на листе {w_sheet} отличается от стандартного - необходимо изучить ошибку!')
                confirmed_diagnosis_all_visits_dfs_list.append(target_df)

            elif w_sheet in common_survey_visits_list:
                print(w_sheet)
                target_df = pd.read_excel(path_to_test_xlsx + _,
                                    sheet_name = w_sheet,
                                    skiprows = 1,
                                    dtype=str)
                cols_quantity = target_df.shape[1]
                print(cols_quantity)
                try:
                    target_df.columns = cols_name_dict[cols_quantity]
                except:
                    print(f'Количество столбцов на листе {w_sheet} отличается от стандартного - необходимо изучить ошибку!')
                common_survey_all_visits_dfs_list.append(target_df)

            else: print(f'Такого наименования листов раньше не встречалось - {w_sheet}, что делать?')
    
all_oncologist_consultation = pd.concat(oncologist_consultations_all_visits_dfs_list, axis=0)
all_oncologist_consultation = all_oncologist_consultation.reset_index(drop = True)

all_confirmed_diagnosis_visits = pd.concat(confirmed_diagnosis_all_visits_dfs_list, axis=0)
all_confirmed_diagnosis_visits = all_confirmed_diagnosis_visits.reset_index(drop = True)

all_common_surveys = pd.concat(common_survey_all_visits_dfs_list, axis=0)
all_common_surveys = all_common_surveys.reset_index(drop = True)


end_reading = dt.now()

print(f'Приемы: первичный, повторный, после ОК, выписка справок - размер таблицы: {all_oncologist_consultation.shape}',
      f'Приемы: в рамках ДН, для направления на лечение, снятие с учета - размер таблицы: {all_confirmed_diagnosis_visits.shape}',
      f'Общий осмтор - размер таблицы: {all_common_surveys.shape}',
      f'\nНачало сборки:{start_reading}',
      f'Конец сборки: {end_reading}',
      sep='\n')

Начало сборки: 2022-05-18 12:49:43.954447
Читение: onko_new_20220514.xlsx
Первичная консультация +
80
Повторная консультация +
210
КО в рамках ДН +
102
Повторная после ОК +
75
Справки и направления +
29
Выписка рецепта на ЛП +
62
Снятие с учета +
22
Общий осмотр
94
Читение: Наименования_столбцов_для_сборки_всех_приемов_онколога.xlsx
Такого наименования листов раньше не встречалось - Sheet1 (3), что делать?
Такого наименования листов раньше не встречалось - кол-ов признаков по типам Прием, что делать?
Такого наименования листов раньше не встречалось - проверка корректности значений, что делать?
Читение: СЛОВАРЬ_ЦАОП_2.xlsx
Такого наименования листов раньше не встречалось - Модель, что делать?
Такого наименования листов раньше не встречалось - Лист1, что делать?
Приемы: первичный, повторный, после ОК, выписка справок - размер таблицы: (979, 262)
Приемы: в рамках ДН, для направления на лечение, снятие с учета - размер таблицы: (305, 143)
Общий осмтор - размер таблицы: (1284, 94)

Начало с

In [79]:
all_oncologist_consultation.shape, all_confirmed_diagnosis_visits.shape, all_common_surveys.shape

((979, 262), (305, 143), (1284, 94))

In [80]:
%%time
# 1. сборка всех видов приемов кроме данных об общем осмотре

pre_total_data = pd.concat([all_oncologist_consultation, all_confirmed_diagnosis_visits])

total_data = pre_total_data.copy()

# удобный мне порядок столбцов
total_data = total_data[[
     'тип приема',
'общая информация - ID протокола',
 'общая информация - ID пациента',
 'общая информация - Дата и время подписания протокола',
 'общая информация - ФИОпациента',
 'общая информация - ДР пациента',
 'общая информация - ОМС пациента',
 'общая информация - ФИО врача',
 'общая информация - Должность врача',
 'общая информация - МО приема',
 'общая информация - Структурное подразделение',
 'общая информация - Филиал',
 'общая информация - Отделение',
 'общая информация - Специализация врача',
 'Прикрепление - ID участка прикрепления в МУ приема',
 'Прикрепление - Участок прикрепления в МУ приема',
 'Прикрепление - Прикрепления в других МО',
 'Анамнез',
 'Жалобы (предъявляет/не предъявляет)',
 'Жалобы',
 'Описание жалоб',
 'Считает себя больным с ',
 'Общее состояние',
 'Уровень сознания',
 'Нервно-психический статус',
 'НПС иное',
 'Состояние пациента (ECOG)',
 'Температура тела',
 'Локальный статус',
 'Направительный диагноз - МКБ-10',
 'Направительный диагноз - Код мкб-10',
 'Диагноз - МКБ-10 расшифровка',
 'Диагноз - Код МКБ-10',
    
 'Диагноз - Развернутый клинический диагноз',
 'Диагноз - Дата устанвоки диагноза',
 'Диагноз - Статус диагноза',
 'Диагноз - Первично-множественная опухоль (да/нет)',
 'Диагноз - Метод подтверждения диагноза',
 'Диагноз - Клиническая группа',
     'Диагноз - Основание диагноза',
    
    'Прочие рекомендации',

        
 'Новый диагноз - МКБ-10',
 'Новый диагноз - Код МКБ-10',
 'Новый диагноз - Дата установления нового диагноза',
 'Новый диагноз - Статус нового диагноза',
 'Единичная опухоль - Сторона поражения',
 'Диагноз - Морфологический тип опухоли',
 'Единичная опухоль - Остоятельства выявления опухоли',
 'Единичная опухоль - Стадия TNM',

 'Единичная опухоль - Классификация остаточной опухоли (R)',
 'Единичная опухоль - Гистопатологическая степень злокачественности (G)',
 'Единичная опухоль - Инвазия в лимфатические сосуды (L)',
 'Единичная опухоль - Венозная инвазия (V)',
 'Единичная опухоль - Периневральная инвазия (Pn)',
 'Единичная опухоль - Локализация отдаленных метастазов',
 'Единичная опухоль - Клиническая группа',
 'Единичная опухоль - Стадия опухолевого процесса',
    
'Биопсия - Требуется взятие биопсии?',
 'Биопсия - Причина, по которой взятие биопсии не требуется',
 'Биопсия - Дата проведения биопсии',
 'Биопсия - Отсутствие информации о дате проведения биопсии',
 'Биопсия - Где проведена биопсия',
 'Биопсия - Где будет проводиться биопсия (тип МО)',
 'Биопсия - Где будет проводиться биопсия (МО)',
 'Биопсия - Отсутствие информации где проведена биопсия',
        
 'Онкоконсилиум - Требуется проведение онкоконсилиума? (true)',
 'Онкоконсилиум - Плановая дата проведения онкоконсилиума',
 'Онкоконсилиум - Место проведения онкоконсилиума',

 'Специализированное лечение - Требуется направление на специализированное лечение',
 'Специализированное лечение - Хирургическое лечение',
 'Специализированное лечение - Место проведения хирургического лечения',
 'Специализированное лечение - Лучевое лечение',
 'Специализированное лечение - Лекарственное лечение',
 'Специализированное лечение - Где будет проводиться лекарственное лечение?',
 'Специализированное лечение - Место проведения лечения',
 'Специализированное лечение - Плановая дата проведения лекарственного лечения',
 'Специализированное лечение - Требуется назначение лекарственной терапии?',
 'Специализированное лечение - МНН/ТН препарата',
 'Специализированное лечение - Дозировка',
    
 'Документы на приеме Справки и направления - Тип документа',
 'Документы на приеме Справки и направления - Инструментальные обследования',
 'Документы на приеме Справки и направления - Лабораторные исследования',
 'Документы на приеме Справки и направления - Консультация специалиста',
 'Документы на приеме Справки и направления - Госпитализация',
 'Документы на приеме Справки и направления - Цель выдачи',

 'Вложенный диагноз на первичном приеме (1) - МКБ-10',
 'Вложенный диагноз на первичном приеме (1) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (1) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (1) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (1) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (1) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (1) - Статус диагноза',
 'Вложенный диагноз на первичном приеме (2) - МКБ-10',
 'Вложенный диагноз на первичном приеме (2) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (2) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (2) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (2) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (2) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (2) - Статус диагноза',
 'Вложенный диагноз на первичном приеме (3) - МКБ-10',
 'Вложенный диагноз на первичном приеме (3) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (3) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (3) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (3) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (3) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (3) - Статус диагноза',
 'Вложенный диагноз на первичном приеме (4) - МКБ-10',
 'Вложенный диагноз на первичном приеме (4) - Код МКБ-10',
 'Вложенный диагноз на первичном приеме (4) - Развернутый клинический диагноз',
 'Вложенный диагноз на первичном приеме (4) - Метод подтверждения диагноза',
 'Вложенный диагноз на первичном приеме (4) - Дата установления диагноза',
 'Вложенный диагноз на первичном приеме (4) - Клиническая группа',
 'Вложенный диагноз на первичном приеме (4) - Статус диагноза',
 
 'Единичная опухоль - cT',
 'Единичная опухоль - cN',
 'Единичная опухоль - cM',
 'Единичная опухоль - pT',
 'Единичная опухоль - pN',
 'Единичная опухоль - pM',
    
    'Множественная опухоль(1) - МКБ-10',
 'Множественная опухоль(1) - Код МКБ-10',
 'Множественная опухоль(1) - Развернутый клинический диагноз',
 'Множественная опухоль(1) - Статус диагноза',
 'Множественная опухоль(1) - Дата установления диагноза',
 'Множественная опухоль(1) - Основание диагноза',
 'Множественная опухоль(1) - Метод подтверждения диагноза',
 'Множественная опухоль(1) - Признак основной опухоли',
 'Множественная опухоль(1) - Сторона поражения',
 'Множественная опухоль(1) - Морфологический тип опухоли',
 'Множественная опухоль(1) - Обстоятельства выявления опухоли',
 'Множественная опухоль(1) - Стадия TNM',
 'Множественная опухоль(1) - pT',
 'Множественная опухоль(1) - pN',
 'Множественная опухоль(1) - pM',
 'Множественная опухоль(1) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(1) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(1) - Периневральная инвазия (Pn)',
 'Множественная опухоль(1) - Венозная инвазия (V)',
 'Множественная опухоль(1) - Инвазия в лимфатические сосуды (L)',
 'Множественная опухоль(1) - Локализация отдаленных метастазов',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.1',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.2',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.3',
 'Множественная опухоль(1) - cT',
 'Множественная опухоль(1) - cN',
 'Множественная опухоль(1) - cM',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.4',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.5',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.6',
 'Множественная опухоль(1) - Локализация отдаленных метастазов.7',
 'Множественная опухоль(1) - Стадия опухолевого процесса',
 'Множественная опухоль(1) - Клиническая группа',
    
 'Множественная опухоль(2) - МКБ-10',
 'Множественная опухоль(2) - Код МКБ-10',
 'Множественная опухоль(2) - Развернутый клинический диагноз',
 'Множественная опухоль(2) - Статус диагноза',
 'Множественная опухоль(2) - Дата установления диагноза',
 'Множественная опухоль(2) - Основание диагноза',
 'Множественная опухоль(2) - Метод подтверждения диагноза',
 'Множественная опухоль(2) - Признак основной опухоли',
 'Множественная опухоль(2) - Сторона поражения',
 'Множественная опухоль(2) - Морфологический тип опухоли',
 'Множественная опухоль(2) - Обстоятельства выявления опухоли',
 'Множественная опухоль(2) - Стадия TNM',
 'Множественная опухоль(2) - pT',
 'Множественная опухоль(2) - pN',
 'Множественная опухоль(2) - pM',
 'Множественная опухоль(2) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(2) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(2) - Периневральная инвазия (Pn)',
 'Множественная опухоль(2) - Венозная инвазия (V)',
 'Множественная опухоль(2) - Инвазия в лимфатические сосуды (L)',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.8',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.9',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.10',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.11',
 'Множественная опухоль(2) - cT',
 'Множественная опухоль(2) - cN',
 'Множественная опухоль(2) - cM',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.12',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.13',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.14',
 'Множественная опухоль(2) - Локализация отдаленных метастазов.15',
 'Множественная опухоль(2) - Стадия опухолевого процесса',
 'Множественная опухоль(2) - Клиническая группа',
 'Множественная опухоль(3) - МКБ-10',
 'Множественная опухоль(3) - Код МКБ-10',
 'Множественная опухоль(3) - Развернутый клинический диагноз',
 'Множественная опухоль(3) - Статус диагноза',
 'Множественная опухоль(3) - Дата установления диагноза',
 'Множественная опухоль(3) - Основание диагноза',
 'Множественная опухоль(3) - Метод подтверждения диагноза',
 'Множественная опухоль(3) - Признак основной опухоли',
 'Множественная опухоль(3) - Сторона поражения',
 'Множественная опухоль(3) - Морфологический тип опухоли',
 'Множественная опухоль(3) - Обстоятельства выявления опухоли',
 'Множественная опухоль(3) - Стадия TNM',
 'Множественная опухоль(3) - pT',
 'Множественная опухоль(3) - pN',
 'Множественная опухоль(3) - pM',
 'Множественная опухоль(3) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(3) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(3) - Периневральная инвазия (Pn)',
 'Множественная опухоль(3) - Венозная инвазия (V)',
 'Множественная опухоль(3) - Инвазия в лимфатические сосуды (L)',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.16',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.17',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.18',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.19',
 'Множественная опухоль(3) - cT',
 'Множественная опухоль(3) - cN',
 'Множественная опухоль(3) - cM',
     'Множественная опухоль(3) - Локализация отдаленных метастазов.20',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.21',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.22',
 'Множественная опухоль(3) - Локализация отдаленных метастазов.23',
 'Множественная опухоль(3) - Стадия опухолевого процесса',
 'Множественная опухоль(3) - Клиническая группа',
 'Множественная опухоль(4) - МКБ-10',
 'Множественная опухоль(4) - Код МКБ-10',
 'Множественная опухоль(4) - Развернутый клинический диагноз',
 'Множественная опухоль(4) - Статус диагноза',
 'Множественная опухоль(4) - Дата установления диагноза',
 'Множественная опухоль(4) - Основание диагноза',
 'Множественная опухоль(4) - Метод подтверждения диагноза',
 'Множественная опухоль(4) - Признак основной опухоли',
 'Множественная опухоль(4) - Сторона поражения',
 'Множественная опухоль(4) - Морфологический тип опухоли',
 'Множественная опухоль(4) - Обстоятельства выявления опухоли',
 'Множественная опухоль(4) - Стадия TNM',
 'Множественная опухоль(4) - pT',
 'Множественная опухоль(4) - pN',
 'Множественная опухоль(4) - pM',
 'Множественная опухоль(4) - Классификация остаточной опухоли (R)',
 'Множественная опухоль(4) - Гистопатологическая степень злокачественности (G)',
 'Множественная опухоль(4) - Периневральная инвазия (Pn)',
 'Множественная опухоль(4) - Венозная инвазия (V)',
 'Множественная опухоль(4) - Инвазия в лимфатические сосуды (L)',
     'Множественная опухоль(4) - Локализация отдаленных метастазов.24',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.25',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.26',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.27',
 'Множественная опухоль(4) - cT',
 'Множественная опухоль(4) - cN',
 'Множественная опухоль(4) - cM',
     'Множественная опухоль(4) - Локализация отдаленных метастазов.28',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.29',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.30',
 'Множественная опухоль(4) - Локализация отдаленных метастазов.31',
 'Множественная опухоль(4) - Стадия опухолевого процесса',
 'Множественная опухоль(4) - Клиническая группа'
]]

Wall time: 18 ms


In [81]:
print('-=< Start >=-')
start_func_wt = dt.now()


# функция которая форматирует данные выгрузки (в том виде в котором мы получаем ее сейчас

# на входе принимает total_data - таблица, которую необходимо обработать
# предобработка описана ниже

############################################################################################



######### НЕОБХОДИМА ПЕРИОДИЧЕСКАЯ ПРОВЕРКА состояния форматов в выгрузке #########
# = '2020-01-15'
first_date_type_list = ['Биопсия - Дата проведения биопсии',
                        'Специализированное лечение - Плановая дата проведения лекарственного лечения']

# = '1997-04-05 00:00:00'
dob = ['общая информация - ДР пациента']

# = '1997-04-05T00:00:00+04:00'
second_date_type_list = ['общая информация - Дата и время подписания протокола',
                         'Считает себя больным с ',
                        'Диагноз - Дата устанвоки диагноза',
                        'Новый диагноз - Дата установления нового диагноза',
                        'Онкоконсилиум - Плановая дата проведения онкоконсилиума',                         
                        'Вложенный диагноз на первичном приеме (1) - Дата установления диагноза',
                        'Вложенный диагноз на первичном приеме (2) - Дата установления диагноза',
                        'Вложенный диагноз на первичном приеме (3) - Дата установления диагноза',
                        'Вложенный диагноз на первичном приеме (4) - Дата установления диагноза',
                        'Множественная опухоль(1) - Дата установления диагноза', 
                        # понять почему остался постфикс в конце
                        'Множественная опухоль(2) - Дата установления диагноза',
                        'Множественная опухоль(3) - Дата установления диагноза',
                        'Множественная опухоль(4) - Дата установления диагноза']

date_cols = first_date_type_list+dob+second_date_type_list

############################################################################################

# цикл обработки дат к формату dt.date

for _ in total_data.columns.tolist():
    if _ in date_cols:
        total_data[_] = total_data[_].fillna(pd.NaT)
        if _ == 'общая информация - ДР пациента':
            total_data[_] = pd.to_datetime(total_data[_], format = '%Y-%m-%d %H:%M:%S', errors='ignore').dt.date
        
        elif _ in second_date_type_list:
            try:
                total_data[_] = total_data[_].str.replace(r'T.+','')
            except:
                print(f'error here:\n{_}')
            
            total_data[_] = pd.to_datetime(total_data[_], format = '%Y-%m-%d', errors='ignore').dt.date
        elif _ in total_data:
            total_data[_] = pd.to_datetime(total_data[_], format = '%Y-%m-%d', errors='ignore').dt.date

total_data = total_data.sort_values(by='общая информация - Дата и время подписания протокола',
                                        ascending=False).copy()
print(f'даты успешно форматированы и отсортированы в порядке убывания Даты приема ("Дата и время подписания протокола")\n')
# 
print(f'подгружает значения ЦАОП...........')
total_data['ЦАОП'] = total_data['общая информация - МО приема'].map(true_caop.set_index('Названия строк')['ЦАОП'])

group_patients_caop_by_id = total_data.groupby('общая информация - ID пациента')
group_patients_caop_by_id = group_patients_caop_by_id.apply(lambda x: x['ЦАОП'].unique())
group_patients_caop_by_id = group_patients_caop_by_id.to_frame()
group_patients_caop_by_id['общая информация - ID пациента'] = group_patients_caop_by_id.index
group_patients_caop_by_id.columns = ['ЦАОП_id', 'общая информация - ID пациента']

group_second = total_data.groupby('общая информация - ОМС пациента')
group_patients_caop_by_oms = group_second.apply(lambda x: x['ЦАОП'].unique())
group_patients_caop_by_oms = group_patients_caop_by_oms.to_frame()
group_patients_caop_by_oms['общая информация - ОМС пациента'] = group_patients_caop_by_oms.index
group_patients_caop_by_oms.columns = ['ЦАОП_ОМС', 'общая информация - ОМС пациента']




total_data_caop = total_data.merge(group_patients_caop_by_id.rename_axis("index"),
                                                                        how='left',
                                                                        left_on='общая информация - ID пациента',
                                                                        right_on='общая информация - ID пациента')

total_data_caop = total_data_caop.merge(group_patients_caop_by_oms.rename_axis("index"),
                                                                        how='left',
                                                                        left_on='общая информация - ОМС пациента',
                                                                        right_on='общая информация - ОМС пациента')



# варварская распаковка всех ЦАОП пациента, если их было несколько за обрабатываемый период

for _ in ['ЦАОП_id', 'ЦАОП_ОМС']:
    total_data_caop[_] = total_data_caop[_].astype(str)
    total_data_caop[_] = total_data_caop[_].str.replace("['","", regex=False)
    total_data_caop[_] = total_data_caop[_].str.replace("']","", regex=False)
    total_data_caop[_] = total_data_caop[_].str.replace("' '",", 2) ", regex=False)


# сравнивает где длиннее строка в ячейке и забирает более полные данные

total_data_caop['ЦАОП_в'] = np.where(len(total_data_caop['ЦАОП_id']) < len(total_data_caop['ЦАОП_ОМС']),
                                                                            total_data_caop['ЦАОП_ОМС'],
                                                                            total_data_caop['ЦАОП_id'])        


print('Кол-во строк по значениям для столбца ЦАОП:',
      f"{total_data_caop['ЦАОП_в'].value_counts()}",
     '\n\nЕсли встречаются неподгруженные значения это нужно разобрать сравнив спрочник с исходником выгрузки!',
     sep='\n')

end_func_wt = dt.now()            
print(f'start_func_work_time: {start_func_wt}\nend_func_work_time: {end_func_wt}')

-=< Start >=-
даты успешно форматированы и отсортированы в порядке убывания Даты приема ("Дата и время подписания протокола")

подгружает значения ЦАОП...........


<ipython-input-81-f2d9dec47a52>:52: FutureWarning: The default value of regex will change from True to False in a future version.
  total_data[_] = total_data[_].str.replace(r'T.+','')


Кол-во строк по значениям для столбца ЦАОП:
ММКЦ Коммунарка ЦАОП      406
ГКОБ 1                    317
МКНЦ им. А.С. Логинова    164
ГКБ им. С.П. Боткина      135
ГКБ им. Д.Д. Плетнёва     131
МГОБ 62                   113
Морозовская ДГКБ           12
ГКБ 52                      6
Name: ЦАОП_в, dtype: int64


Если встречаются неподгруженные значения это нужно разобрать сравнив спрочник с исходником выгрузки!
start_func_work_time: 2022-05-18 12:49:46.267231
end_func_work_time: 2022-05-18 12:49:46.672885


In [84]:
total_data.columns.tolist()

['тип приема',
 'общая информация - ID протокола',
 'общая информация - ID пациента',
 'общая информация - Дата и время подписания протокола',
 'общая информация - ФИОпациента',
 'общая информация - ДР пациента',
 'общая информация - ОМС пациента',
 'общая информация - ФИО врача',
 'общая информация - Должность врача',
 'общая информация - МО приема',
 'общая информация - Структурное подразделение',
 'общая информация - Филиал',
 'общая информация - Отделение',
 'общая информация - Специализация врача',
 'Прикрепление - ID участка прикрепления в МУ приема',
 'Прикрепление - Участок прикрепления в МУ приема',
 'Прикрепление - Прикрепления в других МО',
 'Анамнез',
 'Жалобы (предъявляет/не предъявляет)',
 'Жалобы',
 'Описание жалоб',
 'Считает себя больным с ',
 'Общее состояние',
 'Уровень сознания',
 'Нервно-психический статус',
 'НПС иное',
 'Состояние пациента (ECOG)',
 'Температура тела',
 'Локальный статус',
 'Направительный диагноз - МКБ-10',
 'Направительный диагноз - Код мкб-10'

In [92]:
from transliterate import translit

list_of_eng_cols_to_db = []
for _ in total_data_caop.columns.tolist():
    list_of_eng_cols_to_db.append(translit(_, reversed=True))

In [94]:
db_col_list = [s.replace("'", "") for s in list_of_eng_cols_to_db]
db_col_list

['tip priema',
 'obschaja informatsija - ID protokola',
 'obschaja informatsija - ID patsienta',
 'obschaja informatsija - Data i vremja podpisanija protokola',
 'obschaja informatsija - FIOpatsienta',
 'obschaja informatsija - DR patsienta',
 'obschaja informatsija - OMS patsienta',
 'obschaja informatsija - FIO vracha',
 'obschaja informatsija - Dolzhnost vracha',
 'obschaja informatsija - MO priema',
 'obschaja informatsija - Strukturnoe podrazdelenie',
 'obschaja informatsija - Filial',
 'obschaja informatsija - Otdelenie',
 'obschaja informatsija - Spetsializatsija vracha',
 'Prikreplenie - ID uchastka prikreplenija v MU priema',
 'Prikreplenie - Uchastok prikreplenija v MU priema',
 'Prikreplenie - Prikreplenija v drugih MO',
 'Anamnez',
 'Zhaloby (predjavljaet/ne predjavljaet)',
 'Zhaloby',
 'Opisanie zhalob',
 'Schitaet sebja bolnym s ',
 'Obschee sostojanie',
 'Uroven soznanija',
 'Nervno-psihicheskij status',
 'NPS inoe',
 'Sostojanie patsienta (ECOG)',
 'Temperatura tela',
 

In [ ]:
try:
    sqlite_connection = sqlite3.connect('test_sqlite_python.db')
    
# rewrite
    sqlite_create_table_query = '''CREATE TABLE test_sqlite_db (
                                id INTEGER PRIMARY KEY,
                                name TEXT NOT NULL,
                                email text NOT NULL UNIQUE,
                                joining_date datetime,
                                salary REAL NOT NULL);'''

    cursor = sqlite_connection.cursor()
    print("База данных подключена к SQLite")
    
    cursor.execute(sqlite_create_table_query)
    sqlite_connection.commit()
    print("Таблица SQLite создана")

    cursor.close()

except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)
finally:
    if (sqlite_connection):
        sqlite_connection.close()
        print("Соединение с SQLite закрыто")